In [1]:
# IDEAS:
# Remove names
# stopwords
# custom stopwords
# non-alpha chars
# lowercasing

In [130]:
import pandas as pd
import numpy as np
import spacy
import unidecode
import re

In [3]:
nlp = spacy.load("en_core_web_sm")

In [5]:
# Create a pre-processing pipeline.

In [6]:
full_df = pd.read_csv("../../Data/sem-search_papers_set_with_restored_abstract.csv")

/Users/markberger/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
full_df.fillna("None-placeholder", inplace=True)
full_df.drop(["Unnamed: 0", "Unnamed: 0.1", "page_end", "page_start", "publisher", "issue", "indexed_abstract", 
              "volume", "venue", "doc_type"], inplace=True, axis=1)

In [278]:
CUSTOM_STOPS = {"abstract", "background", "background.", "abstract.", "method", 
                "result", "conclusion", "conclusions", "discussion", "PRON", "registration", "url"}

In [123]:
abstracts_subset = full_df.sample(50).abstract

In [125]:
def remove_whitespace_chars(text):
    return " ".join(text.split())
    

In [201]:
def clean_sentence(sentence):
    
    text = re.sub(r'[^a-zA-Z\']', ' ', sentence)
    
    # remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    return text

In [126]:
# We use unidecode to remove accent characters and generally convert chars to their ASCII variant. Although that is not always good in small 
# amounts of text or where precision is crucial, I believe that for text similarity tasks this is generally an improvement.
abstracts_subset = [remove_whitespace_chars(unidecode.unidecode(a)) for a in abstracts_subset if a != "None-placeholder"]

In [354]:
def has_links(token):
    if any(substring in token for substring in {"link_type", "ref?", "=%", "html", "http", 
                                                "www", ".com", "access_num", ".org", ".edu", "arXiv"}):
        return True
    else:
        return False

In [355]:
def check_for_mostly_numeric_string(token):
    
    int_chars = []
    alpha_chars = []
    
    for ch in token:
        if ch.isnumeric():
            int_chars.append(ch)
        elif ch.isalpha():
            alpha_chars.append(ch)
            
    if len(int_chars) > len(alpha_chars):
        return True
    else:
        return False

In [357]:
def create_text_pipeline(documents):
    
    # Load all the documents into a spaCy pipe.
    docs = list(nlp.pipe(documents))
    cleaned_docs = []
    
    # Lowercase + custom stopwords list + remove one character tokens + remove symbolical and punctuation tokens.
    for doc in docs:
        lowercased_sents_without_stops = []
        for sent in doc.sents:
            lowercased_lemmas_one_sent = []
            for token in sent:
                if not token.pos_ in {"SYM" ,"PUNCT"} \
                and len(token) > 1 \
                and not has_links(token.lower_) \
                and not check_for_mostly_numeric_string(token.lower_) \
                and not re.sub(r'[^\w\s]','',token.lemma_) in CUSTOM_STOPS:
                    lowercased_lemmas_one_sent.append(token.lower_)
                    
            sentence = ' '.join(lowercased_lemmas_one_sent)
            
            lowercased_sents_without_stops.append(sentence)
        
        # Discard sentences with less than 4 words.
        cleaned_docs.append([s for s in lowercased_sents_without_stops if len(s.split()) > 3])
    
    return cleaned_docs

In [358]:
# processed = create_text_pipeline(abstracts_subset)

In [359]:
create_text_pipeline([abstracts_subset[1]])

[['prior studies have demonstrated adverse risk associated with baseline and worsening renal function in acute heart failure but none has modeled the trajectories of change in renal function and impact on outcomes',
  'used linear mixed models of serial measurements of blood urea nitrogen and creatinine to describe trajectories of renal function in patients with acute heart failure and renal dysfunction enrolled in the placebo controlled randomized study of the selective a1 adenosine receptor antagonist rolofylline for patients hospitalized with acute decompensated heart failure and volume overload to assess treatment effect on congestion and renal function study',
  'assessed risk of 180-day mortality and 60-day cardiovascular or renal readmission and used cox regression to determine association between renal trajectories and outcomes',
  'compared with patients alive at days patients who died were older had lower blood pressure and ejection fraction and higher creatinine levels at ba